In [19]:
import sys
if ".." not in sys.path:
    sys.path.append("..")

import tensorflow as tf
from tensorflow import keras

from src.constants import LEN_UCI_MOVES

activation = "relu"
inputs = keras.Input(shape=(70), name="all_input")
board_input = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 64)), name="board_input")(inputs)
misc_input = keras.layers.Lambda(lambda x: tf.slice(x, (0, 64), (-1, 6)), name="misc_input")(inputs)

dense_inputs = keras.layers.concatenate([board_input, misc_input])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(70),
    keras.layers.Dense(1000, activation=activation),
    keras.layers.Dense(1000, activation=activation),
    keras.layers.Dense(1000, activation=activation),
    keras.layers.Dense(LEN_UCI_MOVES, activation="sigmoid")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = keras.models.Model(inputs, dense_output, name="dense_actor")
model.compile(
    optimizer = keras.optimizers.legacy.Adam(learning_rate=0.001),
    loss = keras.losses.CategoricalCrossentropy(),
    metrics = [keras.metrics.CategoricalCrossentropy()],
)

In [17]:
import pandas as pd
import numpy as np

test_file = "../data/split/processed/chess1.pkl"
df = pd.read_pickle(test_file)
X = np.concatenate([np.array(df["obs_board"].values.tolist()), np.array(df["obs_misc"].values.tolist())], axis=1)
Y = tf.one_hot(df["move"].values, depth=LEN_UCI_MOVES).numpy()

In [20]:
model.fit(X, Y)

31250/31250 [==============================] - 212s 7ms/step - loss: 5.1548 - categorical_crossentropy: 5.1548


In [21]:
predictions = model.predict(X)

31250/31250 [==============================] - 51s 2ms/step


In [23]:
accuracy = tf.keras.metrics.CategoricalAccuracy(
    name='binary_accuracy', dtype=None
)
accuracy.update_state(Y, predictions)
accuracy.result().numpy()

0.138257

In [24]:
cross_entropy = tf.keras.metrics.CategoricalCrossentropy(
    name='binary_crossentropy',
    dtype=None,
    from_logits=False,
    label_smoothing=0
)
cross_entropy.update_state(Y, predictions)
cross_entropy.result().numpy()

4.7597413

In [26]:
reference_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(70),
    keras.layers.Dense(1000, activation=activation),
    keras.layers.Dense(1000, activation=activation),
    keras.layers.Dense(1000, activation=activation),
    keras.layers.Dense(LEN_UCI_MOVES, activation="sigmoid")
])
reference_predictions = reference_model.predict(X)

def compute_accuracy(predictions):
    accuracy = tf.keras.metrics.CategoricalAccuracy(
        name='binary_accuracy', dtype=None
    )
    accuracy.update_state(Y, predictions)
    return accuracy.result().numpy()

def compute_crossentropy(predictions):
    cross_entropy = tf.keras.metrics.CategoricalCrossentropy(
        name='binary_crossentropy',
        dtype=None,
        from_logits=False,
        label_smoothing=0
    )
    cross_entropy.update_state(Y, predictions)
    return cross_entropy.result().numpy()

print(compute_accuracy(reference_predictions))
print(compute_crossentropy(reference_predictions))

31250/31250 [==============================] - 49s 2ms/step
0.000449
7.587028
